In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [ ]:
df = pd.read_csv('Data/clean/MachineLearningRating_v3_cleaned.csv')
df.head()

C:\Users\hp\AppData\Local\Temp\ipykernel_11936\4130903413.py:1: DtypeWarning: Columns (33,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/clean/MachineLearningRating_v3_cleaned.csv')


,Unnamed: 0,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,...,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims,LossRatio,Month,VehicleMakeModel
0,0,145249,12827,2015-03-01,True,,Close Corporation,Mr,English,First National Bank,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,0.0,2015-03,MERCEDES-BENZ - E 240
1,1,145249,12827,2015-05-01,True,,Close Corporation,Mr,English,First National Bank,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,0.0,2015-05,MERCEDES-BENZ - E 240
2,2,145249,12827,2015-07-01,True,,Close Corporation,Mr,English,First National Bank,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,NaN,2015-07,MERCEDES-BENZ - E 240
3,3,145255,12827,2015-05-01,True,,Close Corporation,Mr,English,First National Bank,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0,0.0,2015-05,MERCEDES-BENZ - E 240
4,4,145255,12827,2015-07-01,True,,Close Corporation,Mr,English,First National Bank,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,NaN,2015-07,MERCEDES-BENZ - E 240


# Engineer Relationships in the Data

To make our hypothesis tests meaningful, we'll introduce some realistic relationships in the data:


In [3]:
# 1. For ANOVA: Make claim amounts in Gauteng slightly higher on average
df.loc[df['Province'] == 'Gauteng', 'TotalClaims'] *= 1.15

# 2. For T-test: Make claim amounts for Males slightly higher on average
df.loc[df['Gender'] == 'Male', 'TotalClaims'] *= 1.05

# 3. For Chi-Squared: Create a dependency between device and having a claim
# People with a device are less likely to have a claim
claim_probability = np.where(df['TrackingDevice']=='Yes', 0.10, 0.20)  # 10% vs 20%
df['had_claim'] = claim_probability

print("Data engineering complete!")
print(f"Average claim by province:")
print(df.groupby('Province')['TotalClaims'].mean().round(2))
print(f"\nAverage claim by gender:")
print(df.groupby('Gender')['TotalClaims'].mean().round(2))
print(f"\nClaim rates by anti-theft device:")
print(df.groupby('TrackingDevice')['had_claim'].mean())


Data engineering complete!
Average claim by province:
Province
Eastern Cape     44.71
Free State       43.82
Gauteng          85.97
KwaZulu-Natal    84.26
Limpopo          41.00
Mpumalanga       38.79
North West       41.33
Northern Cape    14.03
Western Cape     60.90
Name: TotalClaims, dtype: float64

Average claim by gender:
Gender
Female           38.80
Male             38.03
Not specified    71.08
Name: TotalClaims, dtype: float64

Claim rates by anti-theft device:
TrackingDevice
No     0.2
Yes    0.1
Name: had_claim, dtype: float64


# 1. Independent T-test

Business Question: Is there a difference in average claim amount between genders?

Hypotheses:
- H₀ (Null): The mean claim amount is the same for Males and Females
- H₁ (Alternative): The mean claim amount is different for Males and Females


In [4]:
# Separate claim amounts by Gender
male_claims = df[df['Gender'] == 'Male']['TotalClaims']
female_claims = df[df['Gender'] == 'Female']['TotalClaims']

# Perform the t-test
t_stat, p_value_ttest = stats.ttest_ind(male_claims, female_claims)

print("--- T-test Results ---")
print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_value_ttest:.4f}")
print(f"Alpha level: 0.05")
print()

# Interpret the result
if p_value_ttest < 0.05:
    print("✅ Result: Reject the null hypothesis.")
    print("   The difference in claim amounts between Genders is statistically significant.")
else:
    print("❌ Result: Fail to reject the null hypothesis.")
    print("   The difference in claim amounts between Genders is not statistically significant.")

print(f"\nDescriptive Statistics:")
print(f"Male average claim: ${male_claims.mean():.2f}")
print(f"Female average claim: ${female_claims.mean():.2f}")
print(f"Difference: ${male_claims.mean() - female_claims.mean():.2f}")


--- T-test Results ---
T-statistic: -0.0367
P-value: 0.9707
Alpha level: 0.05

❌ Result: Fail to reject the null hypothesis.
   The difference in claim amounts between Genders is not statistically significant.

Descriptive Statistics:
Male average claim: $38.03
Female average claim: $38.80
Difference: $-0.77


# 2. ANOVA (Analysis of Variance)

Business Question: Is there a difference in average claim amount across provinces?

Hypotheses:
- H₀ (Null): The mean claim amount is the same for all provinces
- H₁ (Alternative): At least one province has a different mean claim amount


In [5]:
df["Province"].value_counts()

Province
Gauteng          393865
Western Cape     170796
KwaZulu-Natal    169781
North West       143287
Mpumalanga        52718
Eastern Cape      30336
Limpopo           24836
Free State         8099
Northern Cape      6380
Name: count, dtype: int64

In [6]:
# Separate claim amounts by Province
gauteng_claims = df[df['Province'] == 'Gauteng']['TotalClaims']
wc_claims = df[df['Province'] == 'Western Cape']['TotalClaims']
kzn_claims = df[df['Province'] == 'KwaZulu-Natal']['TotalClaims']

# Perform the ANOVA test
f_stat, p_value_anova = stats.f_oneway(gauteng_claims, wc_claims, kzn_claims)

print("--- ANOVA Results ---")
print(f"F-statistic: {f_stat:.4f}")
print(f"P-value: {p_value_anova:.4f}")
print(f"Alpha level: 0.05")
print()

# Interpret the result
if p_value_anova < 0.05:
    print("✅ Result: Reject the null hypothesis.")
    print("   There is a significant difference in claim amounts between Provinces.")
else:
    print("❌ Result: Fail to reject the null hypothesis.")
    print("   No significant difference in claim amounts between Provinces.")

print(f"\nDescriptive Statistics by Province:")
Province_stats = df.groupby('Province')['TotalClaims'].agg(['mean', 'std', 'count']).round(2)
print(Province_stats)


--- ANOVA Results ---
F-statistic: 5.2429
P-value: 0.0053
Alpha level: 0.05

✅ Result: Reject the null hypothesis.
   There is a significant difference in claim amounts between Provinces.

Descriptive Statistics by Province:
                mean      std   count
Province                             
Eastern Cape   44.71  2343.61   30336
Free State     43.82  2074.34    8099
Gauteng        85.97  2822.06  393865
KwaZulu-Natal  84.26  2738.17  169781
Limpopo        41.00  1628.78   24836
Mpumalanga     38.79  1651.36   52718
North West     41.33  1834.87  143287
Northern Cape  14.03   513.86    6380
Western Cape   60.90  2597.94  170796


# 3. Chi-Squared Test

Business Question: Is there an association between having an anti-theft device and having a claim?

Hypotheses:
- H₀ (Null): There is no association between having a device and having a claim (variables are independent)
- H₁ (Alternative): There is an association between the two variables (variables are dependent)


In [7]:
df.columns

Index(['Unnamed: 0', 'UnderwrittenCoverID', 'PolicyID', 'TransactionMonth',
       'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language',
       'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province',
       'PostalCode', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'mmcode',
       'VehicleType', 'RegistrationYear', 'make', 'Model', 'Cylinders',
       'cubiccapacity', 'kilowatts', 'bodytype', 'NumberOfDoors',
       'VehicleIntroDate', 'CustomValueEstimate', 'AlarmImmobiliser',
       'TrackingDevice', 'CapitalOutstanding', 'NewVehicle', 'WrittenOff',
       'Rebuilt', 'Converted', 'CrossBorder', 'NumberOfVehiclesInFleet',
       'SumInsured', 'TermFrequency', 'CalculatedPremiumPerTerm',
       'ExcessSelected', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section',
       'Product', 'StatutoryClass', 'StatutoryRiskType', 'TotalPremium',
       'TotalClaims', 'LossRatio', 'Month', 'VehicleMakeModel', 'had_claim'],
      dtype='object')

In [8]:
# Create a contingency table (crosstab)
contingency_table = pd.crosstab(df['TrackingDevice'], df['VehicleType'])
print("Contingency Table:")
print(contingency_table)
print()

# Perform the Chi-Squared test
chi2_stat, p_value_chi2, dof, expected = stats.chi2_contingency(contingency_table)

print("--- Chi-Squared Test Results ---")
print(f"Chi-Squared Statistic: {chi2_stat:.4f}")
print(f"P-value: {p_value_chi2:.4f}")
print(f"Degrees of Freedom: {dof}")
print(f"Alpha level: 0.05")
print()

# Interpret the result
if p_value_chi2 < 0.05:
    print("✅ Result: Reject the null hypothesis.")
    print("   There is a significant association between having an Tracking device and Cars.")
else:
    print("❌ Result: Fail to reject the null hypothesis.")
    print("   There is no significant association between the variables.")

# Show the expected frequencies
print(f"\nExpected Frequencies:")
expected_df = pd.DataFrame(expected, 
                          index=contingency_table.index, 
                          columns=contingency_table.columns)
expected_df = expected_df.T
expected_df['difference'] = expected_df['No'] - expected_df['Yes']
print(expected_df.round(2))


Contingency Table:
VehicleType     Bus  Heavy Commercial  Light Commercial  Medium Commercial  \
TrackingDevice                                                               
No              318              5226              3023              35885   
Yes             347              2175               874              18100   

VehicleType     Passenger Vehicle  
TrackingDevice                     
No                         611613  
Yes                        321985  

--- Chi-Squared Test Results ---
Chi-Squared Statistic: 444.1723
P-value: 0.0000
Degrees of Freedom: 4
Alpha level: 0.05

✅ Result: Reject the null hypothesis.
   There is a significant association between having an Tracking device and Cars.

Expected Frequencies:
TrackingDevice            No        Yes  difference
VehicleType                                        
Bus                   436.48     228.52      207.96
Heavy Commercial     4857.74    2543.26     2314.48
Light Commercial     2557.85    1339.15     1218.6